## Analyzing borrowers’ risk of defaulting.

This project is to prepare a report for a bank’s loan division. We’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Our report will be considered when building a credit scoring of a potential customer. A **credit scoring** is used to evaluate the ability of a potential borrower to repay their loan.

In [1]:
import pandas as pd #import pandas library as 'pd'

In [3]:
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/gdrive')

cs = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset/credit_scoring_eng.csv')
print(cs.head(10)) #read credit score 'credit_scoring_eng.csv' file, and saves it to 'cs' variable

Mounted at /content/gdrive
   children  days_employed  dob_years            education  education_id  \
0         1   -8437.673028         42    bachelor's degree             0   
1         1   -4024.803754         36  secondary education             1   
2         0   -5623.422610         33  Secondary Education             1   
3         3   -4124.747207         32  secondary education             1   
4         0  340266.072047         53  secondary education             1   
5         0    -926.185831         27    bachelor's degree             0   
6         0   -2879.202052         43    bachelor's degree             0   
7         0    -152.779569         50  SECONDARY EDUCATION             1   
8         2   -6929.865299         35    BACHELOR'S DEGREE             0   
9         0   -2188.756445         41  secondary education             1   

       family_status  family_status_id gender income_type  debt  total_income  \
0            married                 0      F    employ

## Data Exploration

**Data Description**
- `children` - number of children
- `days_employed` - employee experience
- `dob_years` - customer's age
- `education` - customer's education level
- `education_id` - identifier of customer's education
- `family_status` - customer's martial status
- `family_status_id` - identifier for customer's martial status
- `gender` - customer'gender
- `income_type` - customer's income type
- `debt` - customer has ever defaulted on a loan
- `total_income` - customer's monthly income
- `purpose` - reason for taking a loan



In [4]:
cs.info() #get information about number of rows and columns from the dataset 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
cs.head(5) #display the first five rows

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


There is data that is a bit problematic
- days_employee which should make how long they worked, but there are days where the value is negative, but for now we don't know why it is negative

In [6]:
cs.describe() #get the data statistic

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


There is something missing in the two columns, namely **days_employed** and **total_income** with the same amount of missing data, namely 2174
Some anomalous values
- Children min -1 and max 20
- days_employed is minus and too many max values

In [8]:
cs_null = cs[cs['days_employed'].isna()]

cs_null.isna().sum()
#Display the dataframe with missing value

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

The missing values is simetric because both column has same the number of null object

In [ ]:
# Mari kita terapkan beberapa kondisi untuk memfilter data dan melihat jumlah baris dalam tabel yang telah difilter.
# Menghitung jumlah dob_years untuk days_employed yang kosong kemudian dibandingkan dengan jumlah dob_years untuk total_income yang kosong
# Apabila memiliki jumlah yang sama berarti bisa dipastikan kalau data tersebut simetris, karena sebelumnya juga memiliki jumlah null object yang sama
dob_de = cs[cs['days_employed'].isna()]['dob_years'].sum()
dob_ti = cs[cs['total_income'].isna()]['dob_years'].sum()

print(dob_de)
print(dob_ti)

cs_null = (cs[cs['days_employed'].isna() & cs['total_income'].isna()])
cs_null

94856
94856


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


<div class="alert alert-warning">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

Kerja bagus! Kamu tidak perlu menggunakan fungsi `print` untuk mencetak nilai dataframe.

</div>

<div class="alert alert-info">
<b>Student answer</b> <a class="tocSkip"></a>

Siap kak.

</div>

<div class="alert alert-success">
<b>Chamdani's comment v.2</b> <a class="tocSkip"></a>

Kerja bagus!

</div>


**Kesimpulan sementara**

[Apakah jumlah baris dalam tabel yang telah difilter sesuai dengan jumlah nilai yang hilang? Kesimpulan apa yang bisa kita buat dari hal ini?]

[Hitung persentase nilai yang hilang jika dibandingkan dengan keseluruhan *dataset*. Apakah nilai yang hilang merupakan bagian data yang cukup besar? Jika demikian, Anda sebaiknya perlu mengisi nilai yang hilang. Untuk melakukannya, pertama-tama kita harus mempertimbangkan apakah data yang hilang bisa jadi disebabkan oleh karakteristik nasabah tertentu, seperti jenis pekerjaan atau yang lainnya. Anda harus memutuskan karakteristik mana yang menurut *Anda* mungkin merupakan penyebabnya. Kedua, kita harus memeriksa apakah ada ketergantungan nilai yang hilang pada nilai indikator lain dengan kolom-kolom yang mengidentifikasikan karakteristik tertentu nasabah.]

[Jelaskan langkah Anda selanjutnya dan bagaimana hubungannya dengan kesimpulan yang Anda buat sejauh ini.]

<div class="alert alert-info">
<b>Student answer</b> <a class="tocSkip"></a>

Jumlah baris dalam tabel sesuai, hal ini memastikan bahwa dua data yang hilang simetris atau berada di baris yang sama
   
Untuk menghitung presentase, kita menggunakan jumlah yang hilang di kolom days_employed dibagi dengan total data

</div>

<div class="alert alert-warning">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

Hebat, Namun cobalah untuk menjelaskan ini secara lebih rinci menggunakan bahasa dan kalimat yang naratif. 

Jelaskan apa yang kamu temukan:
* apakah ada nilai yang aneh atau anomali? 
* bagaimana nilai anomali itu bisa terjadi? 
* apa yang akan kamu lakukan untuk mengatasi anomali tersebut?
* apa hasilnya?

</div>

<div class="alert alert-info">
<b>Student answer</b> <a class="tocSkip"></a>

Nilai anomali terjadi pada jumlah anak yang negatif dan jumlah hari kerja yang negatif
    
nilai anomali bisa terjadi karena salah input dari user
    
untuk mengatasi nilai tersebut  saya akan mengubah nilai negatif menjadi positif

</div>

<div class="alert alert-success">
<b>Chamdani's comment v.2</b> <a class="tocSkip"></a>

Kerja bagus!

</div>

In [ ]:
# Mari kita periksa nasabah yang tidak memiliki data tentang karakteristik yang teridentifikasi dan kolom dengan nilai yang hilang
percentage_missing = len(cs_null)/len(cs)
print(f'persentase data yang hilang adalah {percentage_missing :.2%}')

persentase data yang hilang adalah 10.10%


<div class="alert alert-warning">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

Kerja bagus! sekarang tolong tampilkan dalam bentuk 2 nilai desimal dibelakang koma.

</div>

<div class="alert alert-info">
<b>Student answer</b> <a class="tocSkip"></a>

Siap kak.

</div>

<div class="alert alert-success">
<b>Chamdani's comment v.2</b> <a class="tocSkip"></a>

Kerja bagus!

</div>

In [ ]:
# Periksalah distribusinya
print(cs_null['education'].value_counts(normalize=True).mul(100).round(2).astype(str) + '%' )
print()
print(cs_null['family_status'].value_counts(normalize=True).mul(100).round(2).astype(str) + '%' )
print()
print(cs_null['income_type'].value_counts(normalize=True).mul(100).round(2).astype(str) + '%')
print()
print(cs_null['purpose'].value_counts(normalize=True).mul(100).round(2).astype(str) + '%')

secondary education    64.77%
bachelor's degree      22.82%
SECONDARY EDUCATION     3.08%
Secondary Education     2.99%
some college            2.53%
Bachelor's Degree       1.15%
BACHELOR'S DEGREE       1.06%
primary education       0.87%
SOME COLLEGE            0.32%
Some College            0.32%
PRIMARY EDUCATION       0.05%
Primary Education       0.05%
Name: education, dtype: object

married               56.9%
civil partnership    20.33%
unmarried            13.25%
divorced              5.15%
widow / widower       4.37%
Name: family_status, dtype: object

employee         50.83%
business         23.37%
retiree           19.0%
civil servant     6.76%
entrepreneur      0.05%
Name: income_type, dtype: object

having a wedding                            4.23%
to have a wedding                           3.73%
wedding ceremony                             3.5%
construction of own property                3.45%
housing transactions                         3.4%
buy real estate             

<div class="alert alert-warning">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

Untuk memeriksa distribusi gunakanlah `value_counts` untuk setiap kolom dan tampilkan jumlah setiap kategori dan persentasenya dengan format 2 angka desimal setelah koma.
    
Kurang lebih akan tampak seperti ini:
    
| divorced | 57.51% | 6578 |
| -------- | ------ | ---- |

Setelah kamu melakukannya saya rasa kamu akan menemukan insight baru. Semangat!!
</div>

<div class="alert alert-success">
<b>Chamdani's comment v.2</b> <a class="tocSkip"></a>

Kerja bagus!

</div>

[Deksripsikan temuan Anda di sini.]

**Kemungkinan penyebab hilangnya nilai dalam data**
Persebaran data yang hilang paling banyak adalah mereka yang sedang menempuh secondary education, sudah menikah, karyawan dan tujuannya untuk menikah atau membeli rumah
menurut saya disini ada yang aneh, kebanyakan persebaran data peminjam adalah mereka yang sudah menikah tapi tujuan peminjamannya untuk menikah, tapi ini mungkin bukan user yang sama


[Kemukakan gagasan Anda mengenai kemungkinan alasan hilangnya nilai-nilai tersebut. Apakah menurut Anda mereka hilang secara acak atau apakah terdapat pola-pola tertentu?]
pola yang saya temukan untuk data yang hilang adalah mereka yang tidak memiliki pekerjaan pasti tidak memiliki income, jadi ini adalah kesalahan saat submit


[Mari kita mulai memeriksa apakah nilai yang hilang bersifat acak.]

In [ ]:
# Memeriksa distribusi di seluruh *dataset*
print(cs['education'].value_counts(normalize=True).mul(100).round(2).astype(str) + '%')
print()
print(cs['family_status'].value_counts(normalize=True).mul(100).round(2).astype(str) + '%')
print()
print(cs['income_type'].value_counts(normalize=True).mul(100).round(2).astype(str) + '%')
print()
print(cs['purpose'].value_counts(normalize=True).mul(100).round(2).astype(str) + '%')


secondary education    63.88%
bachelor's degree      21.92%
SECONDARY EDUCATION     3.59%
Secondary Education      3.3%
some college             3.1%
BACHELOR'S DEGREE       1.27%
Bachelor's Degree       1.25%
primary education       1.16%
Some College            0.22%
SOME COLLEGE            0.13%
PRIMARY EDUCATION       0.08%
Primary Education       0.07%
graduate degree         0.02%
Graduate Degree          0.0%
GRADUATE DEGREE          0.0%
Name: education, dtype: object

married              57.51%
civil partnership    19.41%
unmarried            13.07%
divorced              5.55%
widow / widower       4.46%
Name: family_status, dtype: object

employee                       51.66%
business                       23.62%
retiree                        17.91%
civil servant                   6.78%
unemployed                      0.01%
entrepreneur                    0.01%
paternity / maternity leave      0.0%
student                          0.0%
Name: income_type, dtype: object

wedd

<div class="alert alert-warning">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

Untuk memeriksa distribusi gunakanlah `value_counts` untuk setiap kolom dan tampilkan jumlah setiap kategori dan persentasenya dengan format 2 angka desimal setelah koma.
    
Kurang lebih akan tampak seperti ini:
    
| divorced | 57.51% | 6578 |
| -------- | ------ | ---- |

Setelah kamu melakukannya saya rasa kamu akan menemukan insight baru. Semangat!!
</div>

<div class="alert alert-success">
<b>Chamdani's comment v.2</b> <a class="tocSkip"></a>

Kerja bagus!

</div>

**Kesimpulan sementara**

[Apakah distribusi dalam *dataset* yang asli mirip dengan distribusi tabel yang telah difilter? Apa artinya hal tersebut untuk kita?]

distribusi data mirip antara tabel yang difilter dengan tabel yang tidak difilter

[Jika menurut Anda kita belum dapat membuat kesimpulan apa pun, mari kita kembali menyelidiki *dataset* lebih lanjut. Mari pikirkan alasan lain yang dapat menyebabkan data hilang dan periksa apakah kita dapat menemukan pola tertentu yang dapat membuat kita berpikir bahwa hilangnya nilai-nilai tersebut tidak terjadi secara acak. Karena ini merupakan tugas Anda, maka bagian ini adalah opsional.]

In [ ]:
# Periksa penyebab dan pola lain yang dapat mengakibatkan nilai yang hilang
(cs.tail(5))
()
(cs_null.tail(5))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate
21510,2,NaN,28,secondary education,1,married,0,F,employee,0,NaN,car purchase


<div class="alert alert-warning">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

Kerja bagus! Kamu tidak perlu menggunakan fungsi `print` untuk mencetak nilai dataframe.

</div>

<div class="alert alert-success">
<b>Chamdani's comment v.2</b> <a class="tocSkip"></a>

Kerja bagus!

</div>

**Kesimpulan sementara**

[Apakah pada akhirnya kita dapat memastikan bahwa nilai yang hilang adalah suatu kebetulan? Periksa hal lain yang menurut Anda penting di sini.]

<div class="alert alert-danger">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

Cobalah untuk menjelaskan kesimpulan proses dan insight mu secara lebih rinci menggunakan kalimat yang naratif. 

Jelaskan apa yang kamu temukan:
* apakah ada nilai yang aneh atau anomali? 
* bagaimana nilai anomali itu bisa terjadi? 
* apa saja yang kamu lakukan untuk mengatasi anomali tersebut?
* apa yang melandasimu untuk melakukan hal tersebut?
* apa insight awal yang kamu harapkan?
* apa hasil sebenarnya?

</div>

<div class="alert alert-info">
<b>Student answer</b> <a class="tocSkip"></a>

- Children min -1 dan max 20
- days_employed bernilai minus dan nilai maxnya terlalu banyak
    
nilai anomali bisa terjadi karena salah input dari user
    
untuk mengatasi nilai tersebut  saya akan mengubah nilai negatif menjadi positif
    
karena nilai anak dan days_employed tidak mungkin negatif


</div>

In [ ]:
# Periksa pola lainnya - jelaskan pola tersebut
print(sorted(cs['income_type'].unique()))
print(sorted(cs_null['income_type'].unique()))

['business', 'civil servant', 'employee', 'entrepreneur', 'paternity / maternity leave', 'retiree', 'student', 'unemployed']
['business', 'civil servant', 'employee', 'entrepreneur', 'retiree']


<div class="alert alert-warning">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

Untuk pola lainnya sebenarnya kamu sudah melakukan hal ini saat melakukan normalisasi pada data `cs` dan `cs_null`

</div>

<div class="alert alert-success">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

kerja bagus!

</div>


**Kesimpulan**

[Apakah Anda menemukan suatu pola? Bagaimana Anda mendapatkan kesimpulan ini?]

[Jelaskan bagaimana Anda akan mengatasi nilai-nilai yang hilang. Pertimbangkan kategori dengan nilai yang hilang.]

[Rencanakan secara singkat langkah Anda selanjutnya untuk mentransformasi data. Anda mungkin perlu mengatasi berbagai jenis masalah: duplikat, pencatatan yang berbeda, sumber data yang salah, dan nilai yang hilang.]

<div class="alert alert-danger">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

Cobalah untuk menjelaskan kesimpulan proses dan insight mu secara lebih rinci menggunakan kalimat yang naratif. 

Jelaskan apa yang kamu temukan:
* apakah ada nilai yang aneh atau anomali? 
* bagaimana nilai anomali itu bisa terjadi? 
* apa saja yang kamu lakukan untuk mengatasi anomali tersebut?
* apa yang melandasimu untuk melakukan hal tersebut?
* apa insight awal yang kamu harapkan?
* apa hasil sebenarnya?

</div>

## Transformasi data

[Mari kita perhatikan setiap kolom untuk melihat masalah apa yang mungkin dimiliki mereka.]

[Mulailah dengan menghapus duplikat dan memperbaiki data tentang informasi pendidikan jika diperlukan.]

In [ ]:
# Mari kita lihat semua nilai di kolom pendidikan untuk memeriksa ejaan apa yang perlu diperbaiki
print(cs['education'].unique())


["bachelor's degree" 'secondary education' 'Secondary Education'
 'SECONDARY EDUCATION' "BACHELOR'S DEGREE" 'some college'
 'primary education' "Bachelor's Degree" 'SOME COLLEGE' 'Some College'
 'PRIMARY EDUCATION' 'Primary Education' 'Graduate Degree'
 'GRADUATE DEGREE' 'graduate degree']


In [ ]:
# Perbaiki pencatatan jika diperlukan
cs['education'] = cs['education'].str.lower()

In [ ]:
# Periksa semua nilai di kolom untuk memastikan bahwa kita telah memperbaikinya dengan tepat
print(cs['education'].unique())


["bachelor's degree" 'secondary education' 'some college'
 'primary education' 'graduate degree']


<div class="alert alert-warning">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

Hebat, Namun cobalah untuk menjelaskan ini secara lebih rinci menggunakan bahasa dan kalimat yang naratif. 

Jelaskan apa yang kamu temukan:
* apakah ada nilai yang aneh atau anomali? 
* bagaimana nilai anomali itu bisa terjadi? 
* apa yang akan kamu lakukan untuk mengatasi anomali tersebut?
* apa hasilnya?

</div>

<div class="alert alert-danger">
<b>Chamdani's comment v.2</b> <a class="tocSkip"></a>

Hebat, Namun cobalah untuk menjelaskan ini secara lebih rinci menggunakan bahasa dan kalimat yang naratif. 

Jelaskan apa yang kamu temukan:
* apakah ada nilai yang aneh atau anomali? 
* bagaimana nilai anomali itu bisa terjadi? 
* apa yang akan kamu lakukan untuk mengatasi anomali tersebut?
* apa hasilnya?

</div>

[Periksa data kolom `children`]

In [ ]:
# Mari kita lihat distribusi nilai pada kolom `children`
print(cs['children'].unique())
print(cs['children'].value_counts())

[ 1  0  3  2 -1  4 20  5]
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


[Apakah terdapat hal-hal yang aneh di kolom tersebut? Bagaimana mereka bisa terjadi? Buat keputusan tentang apa yang akan Anda lakukan dengan data ini dan jelaskan alasannya.]

Ada beberapa data yang kurang masuk akal, yaitu mereka yang memiliki anak 20 (walaupun ini memungkinkan) dan mereka yang memiliki ada -1, ada 47 orang yang memiliki ada -1 dan ada 76 orang yang memiliki 20 anak, 
yang akan saya lakukan adalah sementara membiarkan yang memiliki ada 20 dan mengubah yang memiliki anak -1 menjadi 1 karena asumsi sementara saya itu adalah salah input nilai

<div class="alert alert-success">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

Kerja bagus! 

</div>

In [ ]:
for column in cs.columns: # Chamdani check
    print("Column name: ", column)
    print("unique value: ", cs[column].unique(),"\n")

Column name:  children
unique value:  [ 1  0  3  2 -1  4 20  5] 

Column name:  days_employed
unique value:  [-8437.67302776 -4024.80375385 -5623.42261023 ... -2113.3468877
 -3112.4817052  -1984.50758853] 

Column name:  dob_years
unique value:  [42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75] 

Column name:  education
unique value:  ["bachelor's degree" 'secondary education' 'some college'
 'primary education' 'graduate degree'] 

Column name:  education_id
unique value:  [0 1 2 3 4] 

Column name:  family_status
unique value:  ['married' 'civil partnership' 'widow / widower' 'divorced' 'unmarried'] 

Column name:  family_status_id
unique value:  [0 1 2 3 4] 

Column name:  gender
unique value:  ['F' 'M' 'XNA'] 

Column name:  income_type
unique value:  ['employee' 'retiree' 'business' 'civil servant' 'unemployed'
 'entrepreneur' 'student' 'paternity / mater

In [ ]:
# [perbaiki data berdasarkan keputusan Anda]
cs['children'] = cs.children.replace(-1,1)

<div class="alert alert-danger">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

### error detected
    
Lihatlah diatas blok kode mu saya memverifikasi nilai unik dari setiap kolom. Perhatikan hasilnya!
**Proses ini berdampak besar karena prose yang keliru** Lihat hasil pengecekan nilai unik setalah cell kode itu dijalankan. Dia akan memberikan nilai 1 dimana2. Ini karena kamu memberikan perintas pada kode untuk mengisi seluruh kolom dengan nilai `1` jika `cs.children == -1`
    
Solusinya kamu bisa menggunakan fungsi `replace` seperti referensi berikut ini:
* https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html

</div>

<div class="alert alert-info">
<b>Student answer</b> <a class="tocSkip"></a>

Siap kak


</div>

In [ ]:
for column in cs.columns: # Chamdani check
    print("Column name: ", column)
    print("unique value: ", cs[column].unique(),"\n")

Column name:  children
unique value:  [ 1  0  3  2  4 20  5] 

Column name:  days_employed
unique value:  [-8437.67302776 -4024.80375385 -5623.42261023 ... -2113.3468877
 -3112.4817052  -1984.50758853] 

Column name:  dob_years
unique value:  [42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75] 

Column name:  education
unique value:  ["bachelor's degree" 'secondary education' 'some college'
 'primary education' 'graduate degree'] 

Column name:  education_id
unique value:  [0 1 2 3 4] 

Column name:  family_status
unique value:  ['married' 'civil partnership' 'widow / widower' 'divorced' 'unmarried'] 

Column name:  family_status_id
unique value:  [0 1 2 3 4] 

Column name:  gender
unique value:  ['F' 'M' 'XNA'] 

Column name:  income_type
unique value:  ['employee' 'retiree' 'business' 'civil servant' 'unemployed'
 'entrepreneur' 'student' 'paternity / maternit

In [ ]:
# Periksa kembali kolom `children` untuk memastikan bahwa semuanya telah diperbaiki

print(cs['children'].value_counts())

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64


<div class="alert alert-success">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

Kerja bagus! 

</div>

[Periksa data dalam kolom the `days_employed`. Pertama-tama pikirkan tentang masalah apa yang mungkin ada pada kolom tersebut, serta pikirkan juga apa yang mungkin ingin Anda periksa dan bagaimana Anda akan melakukannya.]

data days_employed berisi berapa lama nasabah telah bekerja, permasalahannya ada dua yaitu nilai data yang bersifat float(harusnya int) dan nilai data yang negatif(harusnya positif)

In [ ]:
# Temukan data yang bermasalah di `days_employed`, jika memang terdapat masalah, dan hitung persentasenya
print(cs['days_employed'].head(5))
wrong_days_employed = cs.loc[cs['days_employed']<0,'days_employed'].count()
amount_days_employed = cs['days_employed'].count()
percentage_wrong_days_employed =  wrong_days_employed / amount_days_employed
pwde = percentage_wrong_days_employed
print(f'data yang salah ada {pwde:.0%}')
print(cs.loc[cs['days_employed']>(365*1000),'days_employed'].count())

0     -8437.673028
1     -4024.803754
2     -5623.422610
3     -4124.747207
4    340266.072047
Name: days_employed, dtype: float64
data yang salah ada 82%
1729


[Jika jumlah data yang bermasalah tinggi, hal tersebut mungkin disebabkan oleh beberapa masalah teknis. Kami mungkin perlu mengusulkan alasan paling jelas mengapa hal tersebut dapat terjadi dan bagaimana seharusnya data yang benar, mengingat kita tidak dapat menghapus baris yang bermasalah ini.]

data yang bermasalah tinggi, ada 80% data bermasalah dari total data yang ada, kesalahan disini mungkin juga terletak di input atau prosesing datanya yang membuat hasilnya menjadi negatif

In [ ]:
# Atasi nilai yang bermasalah, jika ada
cs.loc[cs['days_employed'] < 0 , 'days_employed' ] = cs.loc[cs['days_employed'] < 0 , 'days_employed' ] * -1
cs.loc[cs['days_employed'] > 20000, 'days_employed'] = (cs['dob_years'] - 19) * 365

<div class="alert alert-warning">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

Hebat, Namun cobalah untuk menjelaskan ini secara lebih rinci menggunakan bahasa dan kalimat yang naratif. 

Jelaskan apa yang kamu temukan:
* apakah ada nilai yang aneh atau anomali? 
* bagaimana nilai anomali itu bisa terjadi? 
* apa yang akan kamu lakukan untuk mengatasi anomali tersebut?
* apa hasilnya?

</div>

<div class="alert alert-success">
<b>Chamdani's comment v.2</b> <a class="tocSkip"></a>

Kerja bagus! 

</div>

In [ ]:
# Periksa hasilnya - pastikan bahwa masalahnya telah diperbaiki
print(cs['days_employed'])

0         8437.673028
1         4024.803754
2         5623.422610
3         4124.747207
4        12410.000000
             ...     
21520     4529.316663
21521    17520.000000
21522     2113.346888
21523     3112.481705
21524     1984.507589
Name: days_employed, Length: 21525, dtype: float64


[Sekarang mari kita lihat usia nasabah dan mengecek apakah terdapat masalah di sana. Sekali lagi, pikirkan tentang kemungkinan kejanggalan apa yang bisa kita temui dalam kolom ini, misalnya angka usia yang tidak masuk akal.]

In [ ]:
# Periksa `dob_years` untuk nilai yang mencurigakan dan hitung persentasenya
print(cs['dob_years'].unique())
print(cs['dob_years'].mean())
print(cs['dob_years'].median())
print(cs[cs['dob_years']<5].head(10))

[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]
43.29337979094077
42.0
      children  days_employed  dob_years            education  education_id  \
99           0   -6935.000000          0  secondary education             1   
149          0    2664.273168          0  secondary education             1   
270          3    1872.663186          0  secondary education             1   
578          0   -6935.000000          0  secondary education             1   
1040         0    1158.029561          0    bachelor's degree             0   
1149         0     934.654854          0  secondary education             1   
1175         0   -6935.000000          0  secondary education             1   
1386         0    5043.219890          0    bachelor's degree             0   
1890         0            NaN          0    bachelor's degree             0   
1898     

[Putuskan apa yang akan Anda lakukan dengan nilai yang bermasalah dan jelaskan alasannya.]


<div class="alert alert-danger">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

Deskripsi ini perlu diupdate karena terdapat kesalahan pada penanganan kolom `children` diatas.
    
cobalah untuk menjelaskan ini secara lebih rinci menggunakan bahasa dan kalimat yang naratif. 

Jelaskan apa yang kamu temukan:
* apakah ada nilai yang aneh atau anomali? 
* bagaimana nilai anomali itu bisa terjadi? 
* apa yang akan kamu lakukan untuk mengatasi anomali tersebut?
* apa hasilnya?

</div>

<div class="alert alert-info">
<b>Student answer</b> <a class="tocSkip"></a>
    
nilai yang bermasalah disini adalah ada usia nasabah 0 tahun, kesalahan data ini kemungkinan juga disebabkan saat input data


</div>

<div class="alert alert-success">
<b>Chamdani's comment v.2</b> <a class="tocSkip"></a>

Kerja bagus! 

</div>

In [ ]:
# Atasi masalah pada kolom `dob_years`, jika terdapat masalah
mean_age = (int(cs['dob_years'].mean()))
cs.loc[cs['dob_years'] < 5,'dob_years'] = mean_age


In [ ]:
# Periksa hasilnya - pastikan bahwa masalahnya telah diperbaiki
print(cs['dob_years'].unique())
print(cs[cs['dob_years']<5].head(10))

[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51 59 29 60 55 58 71
 22 73 66 69 19 72 70 74 75]
Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose, age_category, employed_time, income_level, debt_meaning, purpose_category]
Index: []


[Sekarang saatnya memeriksa kolom `family_status`. Periksalah nilai seperti apa yang dimuat di dalam kolom ini dan masalah apa yang mungkin perlu Anda atasi.]

In [ ]:
# Mari kita lihat nilai untuk kolom ini
print(cs['family_status'].unique())

['married' 'civil partnership' 'widow / widower' 'divorced' 'unmarried']


<div class="alert alert-danger">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

ERROR! data bernilai 1 disini disebabkan karena proses penanganan yang keliru [disini](#error_detected) saran saya hilangkan kode diatas!

</div>

<div class="alert alert-info">
<b>Student answer</b> <a class="tocSkip"></a>

Siap kak


</div>

<div class="alert alert-success">
<b>Chamdani's comment v.2</b> <a class="tocSkip"></a>

Kerja bagus! 

</div>

In [ ]:
# Periksa hasilnya - pastikan nilainya telah diperbaiki
print(cs['family_status'].unique())

['married' 'civil partnership' 'widow / widower' 'divorced' 'unmarried']


[Sekarang saatnya memeriksa kolom `gender`. Periksalah nilai seperti apa yang dimuat di dalam kolom ini dan masalah apa yang mungkin perlu Anda atasi]

In [ ]:
# Mari kita liat nilai dalam kolom ini
print(cs['gender'].unique())


['F' 'M' 'unknown']


<div class="alert alert-danger">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

ERROR! data bernilai 1 disini disebabkan karena proses penanganan yang keliru [disini](#error_detected) 
</div>

<div class="alert alert-info">
<b>Student answer</b> <a class="tocSkip"></a>

Siap kak


</div>

<div class="alert alert-success">
<b>Chamdani's comment v.2</b> <a class="tocSkip"></a>

Kerja bagus! 

</div>

In [ ]:
# Atasi nilai-nilai yang bermasalah, jika ada
cs.loc[cs['gender']=='XNA','gender'] = 'unknown'

<div class="alert alert-danger">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

ERROR! data bernilai 1 disini disebabkan karena proses penanganan yang keliru [disini](#error_detected) saran saya hilangkan kode `cs.loc[cs['gender']==1,'gender'] = 'unknown'`!

</div>

<div class="alert alert-info">
<b>Student answer</b> <a class="tocSkip"></a>

Siap kak


</div>

<div class="alert alert-success">
<b>Chamdani's comment v.2</b> <a class="tocSkip"></a>

Kerja bagus! 

</div>

In [ ]:
# Periksa hasilnya - pastikan bahwa masalahnya telah diperbaiki

print(cs['gender'].unique())

['F' 'M' 'unknown']


<div class="alert alert-warning">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

Hebat, Namun cobalah untuk menjelaskan ini secara lebih rinci menggunakan bahasa dan kalimat yang naratif. 

Jelaskan apa yang kamu temukan:
* apakah ada nilai yang aneh atau anomali? 
* bagaimana nilai anomali itu bisa terjadi? 
* apa yang akan kamu lakukan untuk mengatasi anomali tersebut?
* apa hasilnya?

</div>

[Sekarang saatnya memeriksa kolom `income_type`. Periksalah nilai seperti apa yang dimuat di dalam kolom ini dan masalah apa yang mungkin perlu Anda atasi]

In [ ]:
# Mari kita lihat nilai dalam kolom ini
print(cs['income_type'].unique())
print(cs['income_type'].value_counts())

['employee' 'retiree' 'business' 'civil servant' 'unemployed'
 'entrepreneur' 'student' 'paternity / maternity leave']
employee                       11084
business                        5078
retiree                         3829
civil servant                   1457
unemployed                         2
entrepreneur                       2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64


<div class="alert alert-danger">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

ERROR! data bernilai 1 disini disebabkan karena proses penanganan yang keliru [disini](#error_detected) 
</div>

<div class="alert alert-info">
<b>Student answer</b> <a class="tocSkip"></a>

Siap kak


</div>

<div class="alert alert-success">
<b>Chamdani's comment v.2</b> <a class="tocSkip"></a>

Kerja bagus! 

</div>

In [ ]:
# Atasi nilai yang bermasalah, jika ada

<div class="alert alert-danger">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

ERROR! data bernilai 1 disini disebabkan karena proses penanganan yang keliru [disini](#error_detected) saran saya hilangkan kode diatas!

</div>

<div class="alert alert-info">
<b>Student answer</b> <a class="tocSkip"></a>

Siap kak


</div>

<div class="alert alert-success">
<b>Chamdani's comment v.2</b> <a class="tocSkip"></a>

Kerja bagus! 

</div>

In [ ]:
# Periksa hasilnya - pastikan bahwa masalahnya telah diperbaiki
print(cs['income_type'].unique())
print(cs['income_type'].value_counts())


['employee' 'retiree' 'business' 'civil servant' 'unemployed'
 'entrepreneur' 'student' 'paternity / maternity leave']
employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64


<div class="alert alert-warning">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

Hebat, Namun cobalah untuk menjelaskan ini secara lebih rinci menggunakan bahasa dan kalimat yang naratif. 

Jelaskan apa yang kamu temukan:
* apakah ada nilai yang aneh atau anomali? 
* bagaimana nilai anomali itu bisa terjadi? 
* apa yang akan kamu lakukan untuk mengatasi anomali tersebut?
* apa hasilnya?

</div>

[Sekarang saatnya melihat apakah ada duplikat di dalam data kita. Jika kita menemukannya, Anda harus memutuskan apa yang akan Anda lakukan dengan duplikat tersebut dan menjelaskan alasannya.]

In [ ]:
# Periksa duplikat
print(cs.duplicated().sum())


71


<div class="alert alert-success">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

kerja bagus!

</div>

In [ ]:
# Atasi duplikat, jika ada
cs=cs.drop_duplicates()

<div class="alert alert-success">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

kerja bagus!

</div>

In [ ]:
# Lakukan pemeriksaan terakhir untuk mengecek apakah kita memiliki duplikat
print(cs.duplicated().sum())

0


<div class="alert alert-success">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

kerja bagus!

</div>

In [ ]:
# Periksa ukuran dataset yang sekarang Anda miliki setelah manipulasi pertama yang Anda lakukan
cs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21454 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


<div class="alert alert-success">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

kerja bagus!

</div>

[Jelaskan *dataset* baru Anda: jelaskan secara singkat apa saja perubahannya dan seberapa besar persentase perubahannya, jika ada.]

perubahan yang dilakukan pada dataset adalah untuk mengisi data yang salah pada setiap kolom seperti umur, status pernikahan, jenis income dll. kemudian kita menghapus duplikat yang ada dalam data, data awal adalah 21525 dan data setelah dilakukan pembersihan tersisa 21408

<div class="alert alert-success">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

Kerja bagus!

</div>

# Bekerja dengan nilai yang hilang

[Untuk mempercepat pekerjaan dengan sejumlah data, Anda mungkin ingin menggunakan *dictionary* untuk beberapa nilai yang memiliki ID. Jelaskan mengapa dan *dictionary* apakah yang akan Anda gunakan.]

In [ ]:
# Temukan dictionary

dict_edu = cs.set_index('education_id').to_dict()['education']
dict_fam = cs.set_index('family_status_id').to_dict()['family_status']
print(dict_edu)
print(dict_fam)

{0: "bachelor's degree", 1: 'secondary education', 2: 'some college', 3: 'primary education', 4: 'graduate degree'}
{0: 'married', 1: 'civil partnership', 2: 'widow / widower', 3: 'divorced', 4: 'unmarried'}


<div class="alert alert-danger">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

Kamu bisa gunakan fungsi `zip` dan `dict` dari python untuk melakukannya. Baca [disini](https://cmdlinetips.com/2021/04/convert-two-column-values-from-pandas-dataframe-to-a-dictionary/)

</div>

<div class="alert alert-info">
<b>Student answer</b> <a class="tocSkip"></a>

Siap kak


</div>

<div class="alert alert-success">
<b>Chamdani's comment v.2</b> <a class="tocSkip"></a>

Kerja bagus! 

</div>

### Memperbaiki nilai yang hilang di `total_income`

[Jelaskan secara singkat kolom dengan nilai yang hilang manakah yang perlu Anda tangani. Jelaskan bagaimana Anda akan memperbaikinya.]


[Mulailah dengan mengatasi total nilai pendapatan yang hilang. Buatlah kategori usia untuk nasabah. Buatlah kolom baru yang memuat kategori usia. Strategi ini dapat membantu untuk menghitung total nilai pendapatan.]


In [ ]:
# Mari kita tulis sebuah fungsi untuk menghitung kategori usia

print(cs['dob_years'].min())
print(cs['dob_years'].max())
def age_category(row) :
    age = row['dob_years']
    if age >10 and age <=20 :
        return '11-20'
    elif age >20 and age <=30 :
        return '21-30'
    elif age >30 and age <=40 :
        return '31-40'
    elif age >40 and age <=50 :
        return '41-50'
    elif age >50 and age <=60 :
        return '51-60'
    elif age >60 and age <=70 :
        return '61-70'
    else :
        return '70+'


19
75


<div class="alert alert-success">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

kerja bagus!

</div>

In [ ]:
# Lakukan pengujian untuk melihat apakah fungsi Anda bekerja atau tidak

In [ ]:
# Buatlah kolom baru berdasarkan fungsi
cs['age_category'] = cs.apply(age_category, axis=1)

<div class="alert alert-success">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

kerja bagus!

</div>

In [ ]:
# Periksa bagaimana nilai di dalam kolom baru
print(cs.head())
cs.info()

   children  days_employed  dob_years            education  education_id  \
0         1    8437.673028         42    bachelor's degree             0   
1         1    4024.803754         36  secondary education             1   
2         0    5623.422610         33  secondary education             1   
3         3    4124.747207         32  secondary education             1   
4         0   12410.000000         53  secondary education             1   

       family_status  family_status_id gender income_type  debt  total_income  \
0            married                 0      F    employee     0     40620.102   
1            married                 0      F    employee     0     17932.802   
2            married                 0      M    employee     0     23341.752   
3            married                 0      M    employee     0     42820.568   
4  civil partnership                 1      F     retiree     0     25378.572   

                   purpose age_category  
0    purchase 

<div class="alert alert-success">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

kerja bagus!

</div>

[Pikirkan tentang faktor-faktor yang biasanya bergantung pada pendapatan. Pada akhirnya, Anda akan mengetahui apakah Anda harus menggunakan nilai rata-rata atau median untuk mengganti nilai yang hilang. Untuk membuat keputusan ini, Anda mungkin ingin melihat distribusi faktor-faktor yang Anda identifikasi sebagai dampak dari pendapatan seseorang.]

[Buatlah tabel yang hanya memuat data tanpa nilai yang hilang. Data ini akan digunakan untuk memperbaiki nilai yang hilang.]

In [ ]:
# Buat tabel tanpa nilai yang hilang dan tampilkan beberapa barisnya untuk memastikan semuanya berjalan dengan baik
cs_nonnull = cs.dropna()
print(cs_nonnull.isna().sum())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_category        0
dtype: int64


<div class="alert alert-success">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

kerja bagus!

</div>

In [ ]:
# Perhatikan nilai rata-rata untuk pendapatan berdasarkan faktor yang telah Anda identifikasi
print(cs['total_income'].mean())
print(cs_nonnull['total_income'].mean())
print(cs.mean())
print(cs_nonnull.mean())

26787.568354658677
26787.568354658673
children                0.544327
days_employed        4541.420155
dob_years              43.473665
education_id            0.817097
family_status_id        0.973898
debt                    0.081150
total_income        26787.568355
dtype: float64
children                0.541936
days_employed        4541.420155
dob_years              43.457547
education_id            0.819079
family_status_id        0.972249
debt                    0.081184
total_income        26787.568355
dtype: float64


<div class="alert alert-success">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

kerja bagus!

</div>

In [ ]:
# Perhatikan nilai median untuk pendapatan berdasarkan faktor yang telah Anda identifikasi
print(cs['total_income'].median())
print(cs_nonnull['total_income'].median())
print(cs.median())
print(cs_nonnull.median())


23202.87
23202.87
children                0.000000
days_employed        2188.756445
dob_years              43.000000
education_id            1.000000
family_status_id        0.000000
debt                    0.000000
total_income        23202.870000
dtype: float64
children                0.000000
days_employed        2188.756445
dob_years              43.000000
education_id            1.000000
family_status_id        0.000000
debt                    0.000000
total_income        23202.870000
dtype: float64


<div class="alert alert-success">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

kerja bagus!

</div>

[Ulangi perbandingan tersebut untuk beberapa faktor. Pastikan Anda mempertimbangkan berbagai aspek dan menjelaskan proses berpikir Anda.]

antara mean dan median kedua data yang memiliki nilai hilang dan nilai yang tidak hilang memiliki nilai yang identik.

<div class="alert alert-success">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

kerja bagus!

</div>

[Buatlah keputusan tentang karakteristik yang paling menentukan pendapatan dan apakah Anda akan menggunakan median atau rata-rata. Jelaskan mengapa Anda membuat keputusan ini]
mean dan median memiliki nilai yang tidak terlalu berbeda namun untuk total income kita akan menggunakan nilai median untuk mengisi data yang hilang karena jenis nilai dari total income bisa saja memiliki outlier yang extreme yang akan membuat nilai rata rata tidak akurat

<div class="alert alert-success">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

kerja bagus!

</div>

In [ ]:
#  Tulis fungsi yang akan kita gunakan untuk mengisi nilai yang hilang
median_income_cs = cs_nonnull['total_income'].median() 

def fillna_income(row) :
    row['total_income'] = row['total_income'].fillna(median_income_cs)


<div class="alert alert-success">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

kerja bagus!

</div>

In [ ]:
# Memeriksa bagaimana nilai di dalam kolom baru



In [ ]:
# Terapkan fungsi tersebut ke setiap baris
fillna_income(cs)

<div class="alert alert-success">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

kerja bagus!

</div>

In [ ]:
# Periksa apakah kita mendapatkan kesalahan
print(cs['total_income'].isna().sum())

0


<div class="alert alert-success">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

kerja bagus!

</div>

[Jika Anda menemukan kesalahan dalam menyiapkan nilai untuk data yang hilang, mungkin berarti ada sesuatu yang istimewa tentang data untuk kategori tersebut. Mari pikirkan - Anda mungkin ingin memperbaiki beberapa hal secara manual, jika terdapat cukup data untuk menemukan median/rata-rata.]


In [ ]:
# Ganti nilai yang hilang jika terdapat kesalahan
print(cs['total_income'].unique())

[40620.102 17932.802 23341.752 ... 14347.61  39054.888 13127.587]


<div class="alert alert-success">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

kerja bagus!

</div>

[Setelah Anda selesai dengan `total_income`, periksa apakah jumlah total nilai di kolom ini sesuai dengan jumlah nilai di kolom lain.]

In [ ]:
# Periksa jumlah entri di kolom
cs.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  float64
 11  purpose           21454 non-null  object 
 12  age_category      21454 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.3+ MB


<div class="alert alert-success">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

kerja bagus!

</div>

###  Memperbaiki nilai di `days_employed`

[Pikirkan tentang parameter yang dapat membantu Anda memperbaiki nilai yang hilang di kolom ini. Pada akhirnya, Anda akan mengetahui apakah Anda harus menggunakan nilai rata-rata atau median untuk mengganti nilai yang hilang. Anda mungkin perlu melakukan penelitian yang sama dengan yang Anda lakukan saat memperbaiki data di kolom sebelumnya.]

In [ ]:
# Distribusi median dari `days_employed` berdasarkan parameter yang Anda identifikasi
print(cs['days_employed'].median())



2188.7564450779378


<div class="alert alert-success">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

kerja bagus!

</div>

In [ ]:
# Distribusi rata-rata dari `days_employed` berdasarkan parameter yang Anda identifikasi
print(cs['days_employed'].mean())
print()

4541.420154740021



<div class="alert alert-success">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

kerja bagus!

</div>

[Tentukan apa yang akan Anda gunakan: rata-rata atau median. Jelaskan mengapa.]
menggunakan median karena nilai dari kolom days_employed memiliki nilai yang sangat tinggi dan outlier yang extreme jadi kita memilih nilai tengah dengan menggunakan median

In [ ]:
# Mari tulis fungsi yang menghitung rata-rata atau median (tergantung keputusan Anda) berdasarkan parameter yang Anda identifikasi
median_days_employed = cs['days_employed'].median()
def fill_median_days_employed(row) :
    row['days_employed'] = row['days_employed'].fillna(median_days_employed)

<div class="alert alert-success">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

kerja bagus!

</div>

In [ ]:
# Periksa apakah fungsi Anda dapat bekerja
fill_median_days_employed(cs)


<div class="alert alert-success">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

kerja bagus!

</div>

<div class="alert alert-danger">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

Kamu bisa menghapus kode `fill_median_days_employed(cs)` karena sudah dilakukan diatas:)

</div>

<div class="alert alert-info">
<b>Student answer</b> <a class="tocSkip"></a>

Siap kak


</div>

<div class="alert alert-success">
<b>Chamdani's comment v.2</b> <a class="tocSkip"></a>

Kerja bagus! 

</div>

In [ ]:
# Periksa apakah fungsi Anda bekerja
cs.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21454 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  float64
 11  purpose           21454 non-null  object 
 12  age_category      21454 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.3+ MB


<div class="alert alert-success">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

kerja bagus!

</div>

In [ ]:
# Ganti nilai yang hilang



[Setelah Anda selesai dengan `total_income`, periksa apakah jumlah total nilai di kolom ini sesuai dengan jumlah nilai di kolom lain.]

In [ ]:
# Periksa entri di semua kolom - pastikan kita memperbaiki semua nilai yang hilang

## Pengkategorian Data

[Untuk menjawab pertanyaan dan menguji hipotesis, Anda akan bekerja dengan data yang telah dikategorikan. Lihatlah pertanyaan-pertanyaan yang diajukan kepada Anda dan yang harus Anda jawab. Pikirkan tentang data mana yang perlu dikategorikan untuk menjawab pertanyaan-pertanyaan ini. Di bawah ini, Anda akan menemukan templat yang bisa Anda gunakan untuk mengkategorikan data. Proses pertama mencakup data teks; yang kedua membahas data numerik yang perlu dikategorikan. Anda dapat menggunakan kedua petunjuk yang disarankan atau tidak sama sekali - semuanya terserah Anda.]

[Terlepas dari keputusan Anda untuk mengatasi pengkategorian, pastikan bahwa Anda secara lugas memberikan penjelasan tentang mengapa Anda membuat keputusan tersebut. Ingat: ini adalah pekerjaan Anda dan Andalah yang berhak membuat segala keputusan.]


In [ ]:
cs.head(2)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category,employed_time,income_level,debt_meaning,purpose_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,41-50,Menengah,High Income,Tidak punya utang,property
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,31-40,Singkat,High Income,Tidak punya utang,car


<div class="alert alert-warning">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

Kerja bagus! Kamu tidak perlu menggunakan fungsi `print` untuk mencetak nilai dataframe.

</div>

<div class="alert alert-success">
<b>Chamdani's comment v.2</b> <a class="tocSkip"></a>

Kerja bagus! 

</div>

In [ ]:
print(cs['purpose'].unique())

['purchase of the house' 'car purchase' 'supplementary education'
 'to have a wedding' 'housing transactions' 'education' 'having a wedding'
 'purchase of the house for my family' 'buy real estate'
 'buy commercial real estate' 'buy residential real estate'
 'construction of own property' 'property' 'building a property'
 'buying a second-hand car' 'buying my own car'
 'transactions with commercial real estate' 'building a real estate'
 'housing' 'transactions with my real estate' 'cars' 'to become educated'
 'second-hand car purchase' 'getting an education' 'car'
 'wedding ceremony' 'to get a supplementary education'
 'purchase of my own house' 'real estate transactions'
 'getting higher education' 'to own a car' 'purchase of a car'
 'profile education' 'university education'
 'buying property for renting out' 'to buy a car' 'housing renovation'
 'going to university']


In [ ]:
# Tampilkan nilai data yang Anda pilih untuk pengkategorian
#kategori untuk days_employed
median_days_employed = (cs['days_employed'].median())
parameter_hari = cs['days_employed'].max()/3


def day_employed_category (jumlah_hari) :
    if jumlah_hari >= 0 and jumlah_hari <= parameter_hari :
        return 'Singkat'
    elif jumlah_hari > parameter_hari and jumlah_hari <= (parameter_hari * 2)  :
        return 'Menengah'
    else :
        return 'Lama'
    
#kategori untuk total income
def level_of_income(tingkat_pendapatan):
    if tingkat_pendapatan >= 0 and tingkat_pendapatan <= 1035:
        return 'Low Income'
    elif tingkat_pendapatan >= 1036 and tingkat_pendapatan <= 4045:
        return 'Lower Middle Income'
    elif tingkat_pendapatan >= 4046 and tingkat_pendapatan <=12535: 
        return 'Upper Middle Income'
    elif tingkat_pendapatan >= 12536:
        return 'High Income'
    else :
        return 'unknown'
    
#kategori apakah mempunyai hutang atau tidak
def debt (utang) :
    if utang == 1 :
        return 'Punya utang'
    else :
        return 'Tidak punya utang'
    
def purposecategory(value):
    housing = [x for x in cs['purpose'] if "building" in x or 'real estate' in x or 'housing' in x or 'property' in x or 'house' in x]
    car = [x for x in cs['purpose'] if "car" in x ]
    education = [x for x in cs['purpose'] if "education" in x or 'university' in x or 'educated' in x]
    wedding = [x for x in cs['purpose'] if "wedding" in x ]
    renovation = [x for x in cs['purpose'] if "housing renovation" in x ]

    if value in renovation:
        result = 'renovation'
    elif value in car:
        result = 'car'
    elif value in education:
        result = 'education'
    elif value in wedding:
        result = 'wedding'
    elif value in housing:
        result = 'property'
    else:
        result = 'other'
    return result

<div class="alert alert-danger">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

Untuk fungsi `purposecategory` cobalah menggunakan [List Comprehension](https://www.w3schools.com/python/python_lists_comprehension.asp). Jangan mendefinikan secara manual seperti:
    
</div>

```python
    housing = ['building a property', 'building a real estate', .....,'housing transactions']
    car = ['buying a second-hand car', 'buying my own car','car', .....,'to buy a car','to own a car']
    education = ['education', 'getting an education', ....., 'university education', 'profile education']
    wedding = ['having a wedding','to have a wedding','wedding ceremony']
    renovation = ['housing renovation']
```

<div class="alert alert-success">
<b>Chamdani's comment v.2</b> <a class="tocSkip"></a>

Kerja bagus! 

</div>

In [ ]:
cs['employed_time'] = cs['days_employed'].apply(day_employed_category)
cs['income_level'] = cs['total_income'].apply(level_of_income)
cs['debt_meaning'] = cs['debt'].apply(debt)
cs['purpose_category'] = cs['purpose'].apply(purposecategory)

<div class="alert alert-success">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

kerja bagus!

</div>

[Mari kita memeriksa nilai unik]

In [ ]:
# Periksa nilai unik
print(cs.head(3))

   children  days_employed  dob_years            education  education_id  \
0         1    8437.673028         42    bachelor's degree             0   
1         1    4024.803754         36  secondary education             1   
2         0    5623.422610         33  secondary education             1   

  family_status  family_status_id gender income_type  debt  total_income  \
0       married                 0      F    employee     0     40620.102   
1       married                 0      F    employee     0     17932.802   
2       married                 0      M    employee     0     23341.752   

                 purpose age_category employed_time income_level  \
0  purchase of the house        41-50      Menengah  High Income   
1           car purchase        31-40       Singkat  High Income   
2  purchase of the house        31-40       Singkat  High Income   

        debt_meaning purpose_category  
0  Tidak punya utang         property  
1  Tidak punya utang              car

<div class="alert alert-danger">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

Kerja bagus! Kamu tidak perlu menggunakan fungsi `print` untuk mencetak nilai dataframe.

</div>

[Kelompok utama apakah yang dapat Anda identifikasi berdasarkan nilai uniknya?]

[Kita akan mengkategorikan data kita berdasarkan topik ini.]


In [ ]:
# Mari kita tulis sebuah fungsi untuk mengategorikan data berdasarkan topik umum
print(cs['purpose_category'].unique())

['property' 'car' 'education' 'wedding' 'renovation']


<div class="alert alert-success">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

kerja bagus!

</div>

In [ ]:
# Buat kolom yang memuat kategori dan hitung nilainya


[Jika Anda memutuskan untuk mengategorikan data numerik, Anda juga harus membuat kategori untuk data tersebut.]

In [ ]:
# Lihat semua data numerik di kolom yang Anda pilih untuk pengkategorian


In [ ]:
# Dapatkan kesimpulan statistik untuk kolomnya



[Tentukan rentang apa yang akan Anda gunakan untuk pengelompokan dan jelaskan alasannya.]

In [ ]:
# Buat fungsi yang melakukan pengkategorian menjadi kelompok numerik yang berbeda berdasarkan rentang



In [ ]:
# Buat kolom yang memuat kategori


In [ ]:
# Hitung setiap nilai kategori untuk melihat pendistribusiannya


## Memeriksa hipotesis


**Apakah terdapat korelasi antara memiliki anak dengan melakukan pelunasan tepat waktu?**

In [ ]:
# Periksa data anak dan data pelunasan tepat waktu
print(cs[['children','debt_meaning']].head(3))

# Hitung gagal bayar berdasarkan jumlah anak
pivot_children = pd.pivot_table(cs,index = 'children', columns ='debt' , values='education', aggfunc = 'count')
pivot_children['percentage'] = round(pivot_children[1] / (pivot_children[1]+ pivot_children[0])*100,2)
print(pivot_children)

   children       debt_meaning
0         1  Tidak punya utang
1         1  Tidak punya utang
2         0  Tidak punya utang
debt            0       1  percentage
children                             
0         13028.0  1063.0        7.54
1          4410.0   445.0        9.17
2          1858.0   194.0        9.45
3           303.0    27.0        8.18
4            37.0     4.0        9.76
5             9.0     NaN         NaN
20           68.0     8.0       10.53


In [ ]:
print(pivot_children['percentage'])

children
0      7.54
1      9.17
2      9.45
3      8.18
4      9.76
5       NaN
20    10.53
Name: percentage, dtype: float64


**Kesimpulan**

[Tulis kesimpulan Anda berdasarkan manipulasi dan pengamatan yang Anda lakukan.]

semakin banyak anak yang dimiliki, maka kemungkinan terlambat bayar hutang semakin tinggi

<div class="alert alert-success">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

kerja bagus!

</div>

**Apakah terdapat korelasi antara status keluarga dengan pelunasan tepat waktu?**

In [ ]:
# Periksa data status keluarga dan pelunasan tepat waktu
print(cs[['family_status','debt']].head(3))


# Hitung gagal bayar berdasarkan status keluarga
pivot_family = pd.pivot_table(cs, index = 'family_status' , columns ='debt' , values='education', aggfunc = 'count')
pivot_family['percentage'] = round(pivot_family[1] / (pivot_family[1]+ pivot_family[0])*100,2)
print(pivot_family)


  family_status  debt
0       married     0
1       married     0
2       married     0
debt                   0    1  percentage
family_status                            
civil partnership   3763  388        9.35
divorced            1110   85        7.11
married            11408  931        7.55
unmarried           2536  274        9.75
widow / widower      896   63        6.57


**Kesimpulan**

[Tulis kesimpulan Anda berdasarkan manipulasi dan pengamatan Anda.]

status keluarga tidak memengaruhi telat bayar atau tidak

<div class="alert alert-success">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

kerja bagus!

</div>

**Apakah terdapat korelasi antara tingkat pendapatan dengan membayar kembali tepat waktu?**

In [ ]:
# Periksa data tingkat pendapatan dan pelunasan tepat waktu
print(cs[['income_level','debt']].head(2))
print()

# Hitung gagal bayar berdasarkan tingkat pendapatan
pivot_income = pd.pivot_table(cs, index = 'income_level' , columns ='debt' , values='education', aggfunc = 'count')
pivot_income['percentage'] = round(pivot_income[1] / (pivot_income[1]+ pivot_income[0])*100,2)
print(pivot_income)


  income_level  debt
0  High Income     0
1  High Income     0

debt                     0     1  percentage
income_level                                
High Income          17758  1587        8.20
Lower Middle Income      8     1       11.11
Upper Middle Income   1947   153        7.29


**Kesimpulan**

[Tulis kesimpulan Anda berdasarkan manipulasi dan pengamatan Anda.]
tingkat penghasulan yang rendah kemungkinan besar akan gagal membayar daripada tingkat penghasilan menengah dan tinggi

<div class="alert alert-success">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

kerja bagus!

</div>

**Bagaimana tujuan kredit memengaruhi tingkat gagal bayar?**

In [ ]:
# Periksa persentase tingkat gagal bayar untuk setiap tujuan kredit dan lakukan penganalisisan
pivot_purpose = pd.pivot_table(cs, index = 'purpose_category' , columns ='debt' , values='education', aggfunc = 'count')
pivot_purpose['percentage'] = round(pivot_purpose[1] / (pivot_purpose[1]+ pivot_purpose[0])*100,2)
print(pivot_purpose.sort_values('percentage'))


debt                 0    1  percentage
purpose_category                       
renovation         572   35        5.77
property          8837  692        7.26
wedding           2138  186        8.00
education         3248  319        8.94
car               3903  403        9.36
other             1015  106        9.46


**Kesimpulan**

[Tulis kesimpulan Anda berdasarkan manipulasi dan pengamatan yang Anda lakukan.]

orang yang meminjam uang untuk membeli mobil memiliki tingkat gagal bayar paling besar

<div class="alert alert-success">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

kerja bagus!

</div>

# Kesimpulan umum 

[Tuliskan kesimpulan Anda di bagian akhir ini. Pastikan Anda memasukkan semua kesimpulan penting yang telah Anda buat berkaitan dengan cara Anda memproses dan menganalisis data. Kesimpulan tersebut harus membahas nilai yang hilang, duplikat, dan kemungkinan alasan serta solusi untuk data bermasalah yang harus Anda tangani.]

berdasarkan pengecekan data untuk hipotesis, didapatkan beberapa kesimpulan yaitu
jumlah anak memengaruhi 

[Tuliskan juga kesimpulan Anda mengenai pertanyaan-pertanyaan yang diajukan di sini.]


<div class="alert alert-info">
<b>Student answer</b> <a class="tocSkip"></a>

Berdasarkan hasil pengecekan data diatas
- Jumlah anak menjadi faktor kemungkinan gagal bayar hutang, makin banyak anak makin tinggi kemungkinan tidak bisa bayar hutang
- Penghasilan menengah memiliki persentase gagal bayar lebih tinggi
- Tujuan hutang untuk edukasi dan membeli mobil memiliki kemungkinan gagal bayar yang tinggi


</div>

<div class="alert alert-danger">
<b>Chamdani's comment v.1</b> <a class="tocSkip"></a>

Untuk kesimpulan umum cobalah untuk menjelaskan keseluruhan proses dan insight mu secara lebih rinci menggunakan kalimat yang naratif. 

Jelaskan apa yang kamu temukan:
* apakah ada nilai yang aneh atau anomali? 
* bagaimana nilai anomali itu bisa terjadi? 
* apa saja yang kamu lakukan untuk mengatasi anomali tersebut?
* apa yang melandasimu untuk melakukan hal tersebut?
* apa insight awal yang kamu harapkan?
* apa hasil sebenarnya?

</div>